In [30]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin  
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import holidays

# Creating the Pre-Processor Pipeline: Initial DataFrame

This Pipeline will have 6 steps:

- ColumnSelector: We select the 'counter_id' and 'date' features
- DateFormatter : Separate the datetime column into separate features.<br> We then drop the original 'date' column
- HolidaysFR: Adding a binary feature for french holiday dates and weekends
- EncodeCounter: OneHotEncoding the 'counter_id' feature and then drop the original column.
- MergeWeather: Merging weather data from Orly and Paris
- MergeMultimodal: Merging the average measurements from the multimodal data. 

In [52]:
class ColumnSelector(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[['counter_id','date']]  # , 'coordinates'


class DateFormatter(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        X_copy['date'] = pd.to_datetime(X_copy['date'])
        X_copy['year'] = X_copy['date'].dt.year
        X_copy['month'] = X_copy['date'].dt.month
        #X_copy['week'] = X_copy['date'].dt.isocalendar().week
        X_copy['weekday'] = (X_copy['date'].dt.dayofweek + 1)
        #X_copy['day'] = X_copy['date'].dt.day
        X_copy['hour'] = X_copy['date'].dt.hour
        #X_copy['minute'] = X_copy['date'].dt.minute  # Not relevant
        return X_copy


class HolidaysFR(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        is_holiday = lambda date: 1 if date in holidays.FR() else 0
        is_weekend = lambda day: 1 if day in (6,7) else 0
        X_copy = X.copy()
        X_copy['is_Holiday'] = X_copy['date'].apply(is_holiday)
        X_copy['is_Weekend'] = X_copy['weekday'].apply(is_weekend)
        return X_copy


class EncodeCounter(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        X_copy = pd.get_dummies(X_copy, columns=['counter_id'], dtype=int, drop_first=True)
        return X_copy


class MergeWeather(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        data = pd.read_csv(os.path.join("..", "Datasets", "weather_data_cleaned.csv"))
        data['date'] = pd.to_datetime(data['date']).astype('datetime64[us]')
        merged_data = pd.merge_asof(X, data, on='date')
        #merged_data.drop(columns='date', inplace=True)
        return merged_data

class MergeMultimodal(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X_copy = X.copy()
        # Import Multimodal Data
        mult_df = pd.read_csv(os.path.join("..", "Datasets", "multimodal_data.csv"))
        mult_df['date'] = pd.to_datetime(mult_df['date']).astype('datetime64[us]')
        # Averaging and scaling the count
        mult_df = pd.DataFrame(mult_df.groupby(['date'])['count'].sum()).reset_index()
        scaler = StandardScaler()
        numerical_columns = mult_df.select_dtypes(include='number').columns
        mult_df[numerical_columns] = scaler.fit_transform(mult_df[numerical_columns])
        # Merging data
        merged_data = pd.merge_asof(X_copy, mult_df, on='date')
        merged_data.rename(columns={'count': 'average_multimodal_count'})
        merged_data.drop(columns='date', inplace=True)
        return merged_data


preprocess = Pipeline([
    ("ColumnSelector", ColumnSelector()),
    ("DateFormatter", DateFormatter()),
    ("HolidaysFR", HolidaysFR()),
    ("EncodeCounter", EncodeCounter()),
    ("MergeWeather", MergeWeather()),
    ("MergeMultimodal", MergeMultimodal())
])        
        


We now import and process the dataset.<br>Note: We sort the data by date before separating the target variable. This is required to merge the weather data using pd.merge.asof() to the nearest datetime value later on

In [53]:
import os

df = pd.read_parquet(os.path.join("..", "Datasets", "train.parquet"))
df = df.sort_values('date') # Sort by date 

In [54]:
X = preprocess.fit_transform(df)
y = df['log_bike_count']

In [55]:
X.head()

,year,month,weekday,hour,is_Holiday,is_Weekend,counter_id_100007049-102007049,counter_id_100036718-103036718,counter_id_100036718-104036718,counter_id_100036719-103036719,...,counter_id_100057445-104057445,counter_id_100060178-101060178,counter_id_100060178-102060178,counter_id_100063175-353277233,counter_id_100063175-353277235,counter_id_300014702-353245971,counter_id_300014702-353245972,temperature,precip,count
0,2020,9,2,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-0.01867,-0.155221,-1.046669
1,2020,9,2,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-0.01867,-0.155221,-1.046669
2,2020,9,2,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-0.01867,-0.155221,-1.046669
3,2020,9,2,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,-0.01867,-0.155221,-1.046669
4,2020,9,2,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-0.01867,-0.155221,-1.046669


In [50]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
#from sklearn.model_selection import cross_val_score

# Setting seed
np.random.seed(42)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Linear Regression model
model = LinearRegression()

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
rmse = mean_squared_error(y_test, y_pred, squared=False)

# Print the model coefficients and evaluation metrics
#print("Coefficients:", model.coef_)
#print("Intercept:", model.intercept_)
print("Root Mean Squared Error:", rmse)

Root Mean Squared Error: 1.3783513625581718


### We'll try a random forest regression

In [56]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
#from sklearn.model_selection import cross_val_score

# Setting seed
np.random.seed(42)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Linear Regression model
model = RandomForestRegressor()

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
rmse = mean_squared_error(y_test, y_pred, squared=False)

# Print the model coefficients and evaluation metrics
#print("Coefficients:", model.coef_)
#print("Intercept:", model.intercept_)
print("Root Mean Squared Error:", rmse)

Root Mean Squared Error: 0.4234677446053591


Now, let us train on the entire training set, and predict on the test set

In [8]:
# Fit on the full train set:
X = preprocess.fit_transform(df)
y = df['log_bike_count']

# Initialize the Linear Regression model
model = LinearRegression()

# Fit the model on the training data
model.fit(X,y)

# Import test set
df_test = pd.read_parquet(os.path.join("..", "Datasets", "final_test.parquet"))
df_test = df_test.sort_values('date') # Sort by date 
new_order = df_test.index.tolist() #Keep index order


df_test = preprocess.transform(df_test)
predictions = model.predict(df_test)
predictions_df = pd.DataFrame({'Id': new_order, 'log_bike_count': predictions})
predictions_df = predictions_df.sort_values('Id')

# Specify the file path
csv_file_path = 'submission.csv'

# Write the DataFrame to a CSV file
predictions_df.to_csv(csv_file_path, index=False)


In [57]:
# Fit on the full train set:
X = preprocess.fit_transform(df)
y = df['log_bike_count']

# Initialize the Random Forest model
model = RandomForestRegressor()

# Fit the model on the training data
model.fit(X,y)

# Import test set
df_test = pd.read_parquet(os.path.join("..", "Datasets", "final_test.parquet"))
df_test = df_test.sort_values('date') # Sort by date 
new_order = df_test.index.tolist() #Keep index order


df_test = preprocess.transform(df_test)
predictions = model.predict(df_test)
predictions_df = pd.DataFrame({'Id': new_order, 'log_bike_count': predictions})
predictions_df = predictions_df.sort_values('Id')

# Specify the file path
csv_file_path = 'submission1.csv'

# Write the DataFrame to a CSV file
predictions_df.to_csv(csv_file_path, index=False)


Below i am working on merging the multimodal data by date/location